# Quantum Fourier Transform II

Implementation of the $n$-qubit operation $QFT\left| \psi \right\rangle$ using __[Cirq](https://quantumai.google/cirq)__. The state $\left| \psi \right\rangle$ in this example is given by

\begin{equation}
\left| \psi \right\rangle = \frac{1}{\sqrt{2}}\sum_{j=0}^{15}\sin(2\pi j/16) \left| j \right\rangle,
\end{equation}

and the $QFT$ operator is defined as <a name="ref-1"/>[(Nielsen and Chuang, 2010)](#cite-MikeIke)

\begin{equation}
QFT = \frac{1}{\sqrt{2^n}}\sum_{j=0}^{2^n-1}\sum_{k=0}^{2^n-1}e^{2\pi ikj/{2^n}}\left| k \right\rangle \left\langle j \right|.
\end{equation}

In [26]:
import random
import numpy as np
import cirq
from cirq import H, X, CNOT, S,measure

In [27]:
#QFT gate
#Inputs: quantum circuit and it's qubits
#Output: quantum circuit with QFT applied to first nq qubits
def QFT(qc,qbts,nq):
    
    if nq == 1:
       n = len(qbts) #Number of qubits
    else:
       n = nq #Apply to first nq qubits
    
    #Apply S gates with positive exponents
    for i in range(n):
        qc.append(H(qbts[i])) #Add H gate
        for j in range(i+1,n):
            po = j-i-1
            exp = 1.0/(2**po)
            SS = S**exp
            qc.append(SS(qbts[i]).controlled_by(qbts[j]))
    
    #Swap the qubits
    for i in range(int(n/2)):
        qc.append(cirq.SWAP(qbts[i],qbts[n-1-i]))
        
    return qc

In [30]:
N = 4 #Number of qubits
Ns = 2**N
Final_State = np.zeros(Ns,dtype=complex) #Final state

simulator = cirq.Simulator() #Simulator

count = 0
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                
                #Quantum circuit
                qbs1 = cirq.LineQubit.range(N)
                qc1 = cirq.Circuit()
                if i == 1:
                    qc1.append(X(qbs1[0]))
                
                if j == 1:
                    qc1.append(X(qbs1[1]))
                
                if k == 1:
                    qc1.append(X(qbs1[2]))
                
                if l == 1:
                    qc1.append(X(qbs1[3]))
                
                #Apply QFT 
                qc1 = QFT(qc1,qbs1,N)
            
                #Get state vector before measurement
                Result = simulator.simulate(qc1, qubit_order=[qbs1[0], qbs1[1],qbs1[2],qbs1[3]])
                State = np.around(Result.final_state_vector, 12)
            
                #Add state to final state with appropriate multiplier
                Final_State += np.sin(2.0*np.pi*count/Ns)*State/np.sqrt(2.0)
            
                count+=1

In [31]:
#Final state should be i*sqrt(2)*(|1> - |15>)
for i in range(Ns):
   print(Final_State[i])

0j
(-2.235174181852706e-08+1.414213377982378j)
0j
(-1.1175870923141318e-08+7.450580596923828e-09j)
(2.7755575615628914e-17+0j)
(-1.1175870923141318e-08+7.450580596923828e-09j)
(2.7755575615628914e-17+0j)
(-2.235174181852706e-08-4.0978193283081055e-08j)
(2.7755575615628914e-17+0j)
(2.2351741769954803e-08+4.0978193283081055e-08j)
(2.7755575615628914e-17+0j)
(1.1175870867630167e-08-7.450580596923828e-09j)
(2.7755575615628914e-17+0j)
(1.1175870867630167e-08-7.450580596923828e-09j)
0j
(2.235174176301591e-08-1.414213377982378j)


<!--bibtex

@book{MikeIke,
    title = {Quantum Computation and Quantum Information},
    author = {Nielsen, Michael A. and Chuang, Issac L},
    year = {2010},
    publisher = {Cambridge University Press}
}

-->

# References

<a name="cite-MikeIke"/><sup>[^](#ref-1) </sup>Nielsen, Michael A. and Chuang, Issac L. 2010. _Quantum Computation and Quantum Information_.

